# **Quy trình ETL**

1. **Extract trích xuất dữ liệu**.
      - 1.1 Trích xuất dữ liệu LLM_data từ phần LLM.
      - 1.2 Trích xuất dữ liệu raw_data phần raw data.
2. **Transform dữ liệu**.
      - 2.1 Transform cột article_id từ hai nguồn dữ liệu từ dạng float sang kiểu string.
      - 2.2 Transform join article_id từ hai nguồn dữ liệu raw_data và LLM_data thành merged_data.
      - 2.3 Transform cột price.
          - 2.3.1 Loại bỏ các cột giá Thỏa Thuận và transform cột giá.
      - 2.4 Transform cột area.
          - 2.4.1 Thêm miền chặn dưới của cột area.
      - 2.5 Thêm cột area_per_m2
      - 2.6 Transform cột date_posted từ dạng object sang kiểu datetime64
      - 2.7 Transform cột location sang tọa độ

3. **Load Dữ liệu vào file merged data.csv**.

## **Trích xuất dữ liệu**

#### **Trích xuất dữ liệu từ file raw_data_2_from_post.csv**
File raw_data_2_post là file chứa dữ liệu sau khi đã crawl data từ trang web batdongsan.com.vn.

In [ ]:
import pandas as pd
raw_data_from_post = pd.read_csv('https://raw.githubusercontent.com/KhiemDangLe/Final-Project/main/DataFolder/2_raw_data_from_post.csv')

#### **Trích xuất dữ liệu từ file raw_data_3_from_LLM.csv**
File raw_data_3_from_LLM.csv là file chứa dữ liệu về các thuộc tính của bài đăng được trích xuất từ mô tả của bài đăng nhờ mô hình LLM

In [ ]:
raw_data_extracted_by_LLM = pd.read_csv('https://raw.githubusercontent.com/KhiemDangLe/Final-Project/main/DataFolder/3_raw_data_extracted_by_LLM.csv')

## **Transform dữ liệu**

**raw_data_from_post**

In [ ]:
type(raw_data_from_post['article_id'][1])

In [ ]:
import numpy as np
# sử dụng format_float_positional
for i in range(len(raw_data_from_post['article_id'])):
    raw_data_from_post.loc[i, 'article_id'] = np.format_float_positional(raw_data_from_post.loc[i, 'article_id'], trim='-')
raw_data_from_post['article_id'] = raw_data_from_post['article_id'].astype(str)

**raw_data_extracted_by_LLM**

In [ ]:
type(raw_data_extracted_by_LLM['article_id'][1])

In [ ]:
for i in range(len(raw_data_extracted_by_LLM['article_id'])):
    raw_data_extracted_by_LLM.loc[i, 'article_id'] = np.format_float_positional(raw_data_extracted_by_LLM.loc[i, 'article_id'], trim='-')
raw_data_extracted_by_LLM['article_id'] = raw_data_extracted_by_LLM['article_id'].astype(str)

#### **Join dữ liệu từ hai nguồn raw_data_from_post và raw_data_extracted_by_LLM**

In [ ]:
merged_data = raw_data_from_post.merge(raw_data_extracted_by_LLM, on='article_id', how='inner')
merged_data.drop_duplicates(subset='article_id', inplace=True)

#### **Chuyển đổi cột price**

In [ ]:
import re

# loại bỏ dữ liệu các cột có tên là thỏa thuận
merged_data.drop(index=merged_data[merged_data['price'] == 'Thỏa thuận'].index, inplace=True)

def tach_so_va_don_vi(chuoi):
    # Handle non-string values by converting to string
    if not isinstance(chuoi, str):
        chuoi = str(chuoi)

    # Biểu thức chính quy để tìm các phần số và đơn vị
    pattern = r'([\d.]+)\s*(\D*)'
    
    # Tìm tất cả các cặp số và đơn vị trong chuỗi
    matches = re.findall(pattern, chuoi)
    if matches:
        return matches[0]  # Trả về cặp số và đơn vị đầu tiên (nếu có)
    else:
        return None, None  # Trả về None nếu không tìm thấy

# Extract number and unit before dropping 'price' column
merged_data[['so', 'don_vi']] = merged_data['price'].apply(tach_so_va_don_vi).apply(pd.Series)
merged_data.drop(columns=['price'], inplace=True)

merged_data['don_vi'].unique()
merged_data['don_vi'].str.strip()
merged_data['so'] = merged_data['so'].astype(float)
merged_data.info()

def convert_value(row):
    # Chuyển đổi số thành chuỗi để kiểm tra số chữ số trước dấu thập phân
    so_str = str(row['so'])

    # Tách phần nguyên và phần thập phân của số
    if '.' in so_str:                  # t
        integer_part = so_str.split('.')[0]
    else:
        integer_part = so_str

    # Kiểm tra số chữ số của phần nguyên
    if len(integer_part) < 5:
        if row['don_vi'] == 'tỷ':
            return row['so'] * 1000000000
        elif row['don_vi'] == 'triệu':
            return row['so'] * 1000000
        elif row['don_vi'] == 'nghìn':
            return row['so'] * 1000
        else:
            return row['so']
    else:
        return row['so']


# Áp dụng hàm chuyển đổi cho từng hàng của DataFrame
merged_data['so'] = merged_data.apply(convert_value, axis=1)
merged_data.drop(columns=['don_vi'], inplace=True)
merged_data.rename(columns={'so': 'price'}, inplace=True)

#### **Chuyển đổi cột date_posted từ dạng object sang dạng datetime**

In [ ]:
merged_data['date_posted'] = pd.to_datetime(merged_data['date_posted'], format='%Y-%m-%d')

#### **Thêm cột price_per_m2**

In [ ]:
merged_data['price_per_m2'] = merged_data['price'] / merged_data['area']
merged_data['price_per_m2'].isnull().sum()

### Transform cột location sang tọa độ

In [ ]:
import json
import requests
import time

def get_long_lat(location,api_key):
    url = f"https://geocode.maps.co/search?q={location}&api_key={api_key}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = json.loads(response.text)

        if data:
            longitude = float(data[0]["lon"])
            latitude = float(data[0]["lat"])
            print(f"complete {location}")
            return longitude, latitude
        else:
            return None, None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

def get_long_df(df, start, end,df_2,api):
    for i in range(start, end):
      if not pd.isna(df['street_name'][i]):
        location = df['street_name'][i] + " " + df['district'][i] + " Thành Phố Hồ Chí Minh"
        longitude, latitude = get_long_lat(location,api)
        df_2.at[i, 'longitude'] = longitude
        df_2.at[i, 'latitude'] = latitude
        df_2.at[i,'article_id']= df['article_id'][i]
        time.sleep(1.2)
      else :
        print("11111")
        df_2.at[i, 'longitude'] = None
        df_2.at[i, 'latitude'] = None
        df_2.at[i,'article_id']= df['article_id'][i]
# prompt: đổi kiểu df['street_name'] từ object sang string với cột nào NaN thì để là rỗng

df['street_name'] = df['street_name'].astype(str).fillna("")

df_2= pd.DataFrame()
get_long_df(df,0,len(df),df_2,'API_KEY')
# prompt: lưu df vào một file csv có tên là merged_data_with_long_lat.csv

df_2.to_csv('/content/drive/MyDrive/Data/merged_data_with_long_lat_0_5000.csv', index=False)


## **Load dữ liệu vào file raw_data_4_merged.csv**

In [ ]:
merged_data